In [2]:
import torch
from torch import nn
from torch.nn import functional as F

In [12]:
# nn.Seq 定义了一种特殊的Module
net = nn.Sequential(nn.Linear(20,256), nn.ReLU(),nn.Linear(256,10))
X = torch.rand(2,20)
X

tensor([[0.3166, 0.9682, 0.3694, 0.0946, 0.3295, 0.8017, 0.3254, 0.5405, 0.2912,
         0.0608, 0.7398, 0.9540, 0.5307, 0.2023, 0.2880, 0.3806, 0.4439, 0.2250,
         0.2427, 0.1453],
        [0.4218, 0.2350, 0.3574, 0.6885, 0.6510, 0.9384, 0.5772, 0.6034, 0.5396,
         0.4638, 0.5595, 0.1385, 0.5175, 0.4123, 0.9076, 0.4520, 0.6383, 0.7439,
         0.9566, 0.2347]])

In [13]:
net(X)

tensor([[-0.1470, -0.0309, -0.0689,  0.1621, -0.0551, -0.0288, -0.0168,  0.1333,
         -0.0917,  0.1271],
        [-0.2519, -0.0891, -0.0203,  0.0949, -0.1404, -0.1109,  0.0479,  0.1667,
         -0.1182, -0.0222]], grad_fn=<AddmmBackward0>)

In [14]:
# 自定义块
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256, 10)
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [18]:
X

tensor([[0.3166, 0.9682, 0.3694, 0.0946, 0.3295, 0.8017, 0.3254, 0.5405, 0.2912,
         0.0608, 0.7398, 0.9540, 0.5307, 0.2023, 0.2880, 0.3806, 0.4439, 0.2250,
         0.2427, 0.1453],
        [0.4218, 0.2350, 0.3574, 0.6885, 0.6510, 0.9384, 0.5772, 0.6034, 0.5396,
         0.4638, 0.5595, 0.1385, 0.5175, 0.4123, 0.9076, 0.4520, 0.6383, 0.7439,
         0.9566, 0.2347]])

In [17]:
net = MLP()
net(X)

tensor([[-0.0658,  0.1808,  0.1903,  0.0755,  0.2653,  0.2252, -0.0238,  0.0813,
          0.2426,  0.1836],
        [-0.0763,  0.0854,  0.1400,  0.1076,  0.1872,  0.3128,  0.1978,  0.2524,
          0.3605,  0.0981]], grad_fn=<AddmmBackward0>)

In [19]:
# 顺序块
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
    def forward(self, X):
        # keys() 方法用于返回字典中的所有键（key）
        # values() 方法用于返回字典中所有键对应的值（value）
        # items() 用于返回字典中所有的键值对（key-value）
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20,256), nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[ 0.0895,  0.0291,  0.1111, -0.3493, -0.1170, -0.2064, -0.1268, -0.0166,
          0.0090,  0.1427],
        [ 0.0275,  0.0377,  0.0821, -0.4318, -0.1655, -0.1825, -0.0848, -0.0070,
          0.1432,  0.2138]], grad_fn=<AddmmBackward0>)

In [21]:
# 在正向传播函数中执行代码
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight=torch.rand((20,20), requires_grad=False)
        self.linear=nn.Linear(20,20)
        
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
net = FixedHiddenMLP()
net(X)

tensor(0.1920, grad_fn=<SumBackward0>)

In [22]:
# 混合搭配各种组合块的方法
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16,20),FixedHiddenMLP())
chimera(X)

tensor(0.7425, grad_fn=<SumBackward0>)

In [23]:
# 参数管理
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8,1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.1800],
        [-0.1219]], grad_fn=<AddmmBackward0>)

In [24]:
# 参数访问
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.2653, -0.0606,  0.1381,  0.3183, -0.1676,  0.0995, -0.1830, -0.0293]])), ('bias', tensor([0.1201]))])


In [29]:
# 访问目标参数
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
print(net[2].bias.data.item())

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1201], requires_grad=True)
tensor([0.1201])
0.12005046010017395


In [31]:
net[2].weight.grad == None,net[2].bias.grad == None,

(True, True)

In [35]:
# 一次性访问所有参数
# 列表解析表达式
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])
print((name, param.shape) for name, param in net.named_parameters())

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))
<generator object <genexpr> at 0x000001E4E6063270>


In [36]:
net.state_dict()['2.bias'].data

tensor([0.1201])

In [38]:
def block1():
    return nn.Sequential(nn.Linear(4,8), nn.ReLU(), nn.Linear(8,4),
                          nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}',block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4,1))
rgnet(X)

tensor([[-0.1104],
        [-0.1103]], grad_fn=<AddmmBackward0>)

In [39]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [46]:
# 内置初始化
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
        
net.apply(init_normal)
net[0].weight.shape # 注意第一层的输入和输出，已经W的shape
#net[0].weight.data[0], net[0].bias.data[0]

torch.Size([8, 4])

In [49]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
        
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [50]:
# 对某些块应用不同的初始化方法
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.5439, -0.2019,  0.4158,  0.6409])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [52]:
def my_init(m):
    if type(m) == nn.Linear:
        print(
            "Init",
            *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >=5
        
net.apply(my_init)
# net[0].weight[:2]
net[0].weight

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


Parameter containing:
tensor([[-6.7520, -0.0000, -9.8882,  0.0000],
        [-8.5667, -0.0000, -0.0000,  0.0000],
        [-0.0000,  0.0000, -5.3686,  0.0000],
        [-8.4213, -0.0000,  0.0000,  0.0000],
        [ 0.0000, -0.0000, -9.9321,  0.0000],
        [-0.0000,  6.2530, -0.0000, -5.5379],
        [ 6.0817, -8.8288, -8.9956,  0.0000],
        [ 7.0724,  8.6654,  9.9538,  0.0000]], requires_grad=True)

In [53]:
# 直接更改参数
net[0].weight.data[:] += 1
net[0].weight.data[0,0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000, -8.8882,  1.0000])

In [55]:
# 参数绑定
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared,
                    nn.ReLU(), nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0,0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
